In [ ]:
import cv2
from cv2 import aruco
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
image_path = 'IMG_4171.jpg'

image = cv2.imread(image_path)

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

## Potentially we can use thresholding for aruco recognition, 
## but it work fine even of raw image

# gray = cv2.medianBlur(gray, 5)
# gray = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,91,5)
# gray = sharp.astype(np.uint8)

aruco_dict = aruco.Dictionary_get(aruco.DICT_ARUCO_ORIGINAL)
parameters =  aruco.DetectorParameters_create()
corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
image_markers = aruco.drawDetectedMarkers(image.copy(), corners, ids)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image_markers)
for i in range(len(ids)):
    c = corners[i][0]
    plt.plot([c[0, 0]], [c[0, 1]], "o", label = "id={0}".format(ids[i]))
plt.legend()
plt.show()

In [ ]:
# Coordinates in the space of original image

pts1 = np.array([[151, 128], [252, 4563], [3205, 4564], [3307, 0]]).astype(np.float32)

In [ ]:
# Coordinates in the space of photo
# Last point is the top right corner, measured manually 

pts2 = np.stack([corners[list(ids.T[0]).index(125)][0][0], corners[list(ids.T[0]).index(75)][0][2], 
                 corners[list(ids.T[0]).index(18)][0][2], [2065, 218]]).astype(np.float32)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image_markers)
plt.scatter(pts2[:, 0], pts2[:, 1])
plt.show()

In [ ]:
image_orig_size = (3307, 4677)
M = cv2.getPerspectiveTransform(pts2,pts1)
dst = cv2.warpPerspective(image_markers, M, image_orig_size)

plt.figure(figsize=(12, 12))
plt.subplot(121),plt.imshow(image_markers),plt.title('Input')
plt.subplot(122),plt.imshow(dst),plt.title('Output')
plt.show()

In [ ]:
from matplotlib.patches import RegularPolygon
plt.figure(figsize=(12, 12))

gray = dst
gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray, 13)
gray = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,91,5)
gray = gray.astype(np.uint8)

plt.imshow(gray, cmap='Greys_r')

# Coordinates of top left hexagon center
y = 1174
x = 235 

# Hexagon radius
r = 75

# Hexagon perpendicular
h = 65.25

'''       ____
         /    \r      
        /      \
        \  h|  /
         \__|_/'''

for col_num in range(27):
    for row_num in range(26):
        hexagon = RegularPolygon((x + r * 1.5 * col_num, 1174 + row_num * h * 2 + (col_num % 2) * h), numVertices=6, 
                                 radius=r, alpha=0.2, edgecolor='k', orientation=np.pi/2)
        plt.gca().add_patch(hexagon)

plt.scatter([x], [y])